# Sampling Request Patterns

In [82]:
def sample_requests(all_requests_base, test_fraction):
    # TO ADD: a way of ensuring that we are only taking the first half of observations
    # - Split the observations up into those that are originally in the scheduler
    #   (i.e. those that are added in the upfront bulk request injection)
    #   and those that are injected later.
    #   Only test on those that are in the upfront bulk injection?
    sample_number = round(test_fraction * len(all_requests_base))
    sampled_requests = random.sample(list(all_requests_base.keys()), k=sample_number)
    return sampled_requests

test_fraction = 0.3
sample_requests(all_requests_base, test_fraction)

['30',
 '41',
 '51',
 '89',
 '68',
 '46',
 '32',
 '21',
 '73',
 '72',
 '48',
 '15',
 '29',
 '44',
 '6',
 '54',
 '19',
 '45',
 '77',
 '4',
 '28',
 '36',
 '78',
 '66',
 '87',
 '38',
 '79',
 '76',
 '62',
 '0']

# Estimating average number of requests
per telescope, per night (biased to 2m data)

In [16]:
import json
import statistics as stats

In [2]:
data = json.load(open("data/observation_patterns_v1.json", "r"))

In [19]:
total_count = 0
total_time = 0
time_list = []
for i in data:
    total_count += data[i]["count"]
    current_time = 0
    for t in data[i]["pattern"]:
        current_time += float(t)
    total_time += current_time * data[i]["count"]
    for x in range(data[i]["count"]):
        time_list.append(current_time)
avg_time = total_time/total_count
median_time = stats.median(time_list)
print(total_time, total_count, avg_time, median_time)

3913285.0 2459 1591.4131760878406 900.0


In [20]:
night_length = 12*60*60
num_requests_mean = night_length / avg_time
num_requests_median = night_length / median_time
print(num_requests_mean, num_requests_median)

27.14568450802842 48.0


In [21]:
num_requests_median * 23 # Total requests per night for the entire LCO network?

1104.0

# Modifying Availability Windows

In [83]:
def modify_availability_window(request_data, window_increase):
    request_data = deepcopy(request_data)
    wgroup = request_data["windows"]
    for resource, windows in wgroup.items():
        new_windows = []
        for w in windows:
            new_windows.append(extend_window(w, window_increase))
        wgroup[resource] = new_windows
    request_data["windows"] = wgroup
    return request_data

In [84]:
def extend_window(window, amount, extend_type="right"):
    start = window["start"]
    end = window["end"]
    duration = end-start
    adjustment = int(duration * amount)

    if extend_type == "right":
        end += adjustment
    elif extend_type == "left":
        start -= adjustment
    elif extend_type == "center":
        start -= int(adjustment/2)
        end += int(adjustment/2)

    return {"start": start, "end": end}

In [85]:
modified_requests = deepcopy(all_requests_base)
modified_requests['0'] = modify_availability_window(modified_requests['0'], 2.0)

In [86]:
all_requests_base['0']

{'windows': {'telescope_0': [{'start': 91722, 'end': 109763}]},
 'duration': 2240,
 'proposal': 'proposal_3',
 'resID': 0}

In [87]:
modified_requests['0']

{'windows': {'telescope_0': [{'start': 91722, 'end': 145845}]},
 'duration': 2240,
 'proposal': 'proposal_3',
 'resID': 0}

In [88]:
modified_requests

{'0': {'windows': {'telescope_0': [{'start': 91722, 'end': 145845}]},
  'duration': 2240,
  'proposal': 'proposal_3',
  'resID': 0},
 '1': {'windows': {'telescope_0': [{'start': 66057, 'end': 71472}]},
  'duration': 600,
  'proposal': 'proposal_1',
  'resID': 1},
 '2': {'windows': {'telescope_0': [{'start': 186995, 'end': 237632}]},
  'duration': 6490,
  'proposal': 'proposal_2',
  'resID': 2},
 '3': {'windows': {'telescope_0': [{'start': 213244, 'end': 226336}]},
  'duration': 2260,
  'proposal': 'proposal_4',
  'resID': 3},
 '4': {'windows': {'telescope_0': [{'start': 220701, 'end': 226464}]},
  'duration': 1520,
  'proposal': 'proposal_2',
  'resID': 4},
 '5': {'windows': {'telescope_0': [{'start': 131294, 'end': 131408}]},
  'duration': 60,
  'proposal': 'proposal_4',
  'resID': 5},
 '6': {'windows': {'telescope_0': [{'start': 167083, 'end': 168787}]},
  'duration': 600,
  'proposal': 'proposal_3',
  'resID': 6},
 '7': {'windows': {'telescope_0': [{'start': 243440, 'end': 247453}]}

# CPSAT Interval Variables

In [2]:
import collections
from ortools.sat.python import cp_model

In [6]:
jobs_data = [  # task = (machine_id, processing_time).
    [(0, 3), (1, 2), (2, 2)],  # Job0
    [(0, 2), (2, 1), (1, 4)],  # Job1
    [(1, 4), (2, 3)],  # Job2
]

machines_count = 1 + max(task[0] for job in jobs_data for task in job)
all_machines = range(machines_count)
# Computes horizon dynamically as the sum of all durations.
horizon = sum(task[1] for job in jobs_data for task in job)

In [7]:
model = cp_model.CpModel()

In [8]:
# Named tuple to store information about created variables.
task_type = collections.namedtuple("task_type", "start end interval")
# Named tuple to manipulate solution information.
assigned_task_type = collections.namedtuple(
    "assigned_task_type", "start job index duration"
)

# Creates job intervals and add to the corresponding machine lists.
all_tasks = {}
machine_to_intervals = collections.defaultdict(list)

for job_id, job in enumerate(jobs_data):
    for task_id, task in enumerate(job):
        machine, duration = task
        suffix = f"_{job_id}_{task_id}"
        start_var = model.NewIntVar(0, horizon, "start" + suffix)
        end_var = model.NewIntVar(0, horizon, "end" + suffix)
        interval_var = model.NewIntervalVar(
            start_var, duration, end_var, "interval" + suffix
        )
        all_tasks[job_id, task_id] = task_type(
            start=start_var, end=end_var, interval=interval_var
        )
        machine_to_intervals[machine].append(interval_var)

In [9]:
machine_to_intervals

defaultdict(list,
            {0: [interval_0_0(start = start_0_0, size = 3, end = end_0_0),
              interval_1_0(start = start_1_0, size = 2, end = end_1_0)],
             1: [interval_0_1(start = start_0_1, size = 2, end = end_0_1),
              interval_1_2(start = start_1_2, size = 4, end = end_1_2),
              interval_2_0(start = start_2_0, size = 4, end = end_2_0)],
             2: [interval_0_2(start = start_0_2, size = 2, end = end_0_2),
              interval_1_1(start = start_1_1, size = 1, end = end_1_1),
              interval_2_1(start = start_2_1, size = 3, end = end_2_1)]})

In [10]:
# Create and add disjunctive constraints.
for machine in all_machines:
    model.AddNoOverlap(machine_to_intervals[machine])

# Precedences inside a job.
for job_id, job in enumerate(jobs_data):
    for task_id in range(len(job) - 1):
        model.Add(
            all_tasks[job_id, task_id + 1].start >= all_tasks[job_id, task_id].end
        )

In [11]:
# Makespan objective.
obj_var = model.NewIntVar(0, horizon, "makespan")
model.AddMaxEquality(
    obj_var,
    [all_tasks[job_id, len(job) - 1].end for job_id, job in enumerate(jobs_data)],
)
model.Minimize(obj_var)

In [12]:
solver = cp_model.CpSolver()
status = solver.Solve(model)

In [13]:
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print("Solution:")
    # Create one list of assigned tasks per machine.
    assigned_jobs = collections.defaultdict(list)
    for job_id, job in enumerate(jobs_data):
        for task_id, task in enumerate(job):
            machine = task[0]
            assigned_jobs[machine].append(
                assigned_task_type(
                    start=solver.Value(all_tasks[job_id, task_id].start),
                    job=job_id,
                    index=task_id,
                    duration=task[1],
                )
            )

    # Create per machine output lines.
    output = ""
    for machine in all_machines:
        # Sort by starting time.
        assigned_jobs[machine].sort()
        sol_line_tasks = "Machine " + str(machine) + ": "
        sol_line = "           "

        for assigned_task in assigned_jobs[machine]:
            name = f"job_{assigned_task.job}_task_{assigned_task.index}"
            # Add spaces to output to align columns.
            sol_line_tasks += f"{name:15}"

            start = assigned_task.start
            duration = assigned_task.duration
            sol_tmp = f"[{start},{start + duration}]"
            # Add spaces to output to align columns.
            sol_line += f"{sol_tmp:15}"

        sol_line += "\n"
        sol_line_tasks += "\n"
        output += sol_line_tasks
        output += sol_line

    # Finally print the solution found.
    print(f"Optimal Schedule Length: {solver.ObjectiveValue()}")
    print(output)
else:
    print("No solution found.")

Solution:
Optimal Schedule Length: 11.0
Machine 0: job_1_task_0   job_0_task_0   
           [0,2]          [2,5]          
Machine 1: job_2_task_0   job_0_task_1   job_1_task_2   
           [0,4]          [5,7]          [7,11]         
Machine 2: job_1_task_1   job_2_task_1   job_0_task_2   
           [2,3]          [4,7]          [7,9]          



# Schedule Simulator Outputting Scheduler Info

In [1]:
from ScheduleSimulator import SchedulerSimulation
from scheduler_gurobi import SchedulerGurobi
import json

In [5]:
data = json.load(open("data/performance_input/performanceTest1_100Requests_0.json", "r"))
sim = SchedulerSimulation(data=data)
sim

In [6]:
next_events = sim.get_next_events()
next_events

[0]

In [7]:
sim.process_event_group(next_events)

In [8]:
requests, resources = sim.check_occupied_requests()

In [9]:
info = sim.get_scheduler_info()
info.keys()

dict_keys(['now', 'horizon', 'slice_size', 'resources', 'proposals', 'requests', 'timelimit'])

In [10]:
sched = SchedulerGurobi(info["now"], info["horizon"], info["slice_size"], resources, info["proposals"], requests,
                        verbose=0, timelimit=info["timelimit"], scheduler_type="gurobi")

Scheduling with gurobi solver...


In [11]:
sched.run().keys()

dict_keys(['scheduled', 'now'])

---

In [12]:
import numpy as np

In [13]:
a = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [14]:
np.array_split(a, 1)

[array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10])]

In [15]:
np.array_split(a, 2)

[array([1, 2, 3, 4, 5]), array([ 6,  7,  8,  9, 10])]